# Data augmentation in NLP

In [1]:
text = """Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine."""

# 1 Lexical substitution based on a masked language model (BERT)
Modifying a random word.  
We can then test something more complex based on POS to change some types of words and not others that might be important.

In [2]:
import random

In [3]:
from transformers import pipeline

In [4]:
camembert_fill_mask = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base", top_k=7)

In [5]:
%%time

word_to_change = random.randint(0, len(text.replace(",","").split())-1)
new_text = []
for word in range(len(text.split())):
    if word == word_to_change :
        new_text.append(text.split()[word].replace(text.split()[word_to_change],'<mask>'))
    else :
        new_text.append(text.split()[word])
results = camembert_fill_mask(' '.join(new_text))

# original text
print(text)

# new text
if all(isinstance(elem, list) for elem in results):
    # Sometimes the output from the pipeline is a single list containing the requested top_k and sometimes the pipeline returns a list of lists.
    # So we must handle the two different cases.
    if text.replace("\n","") == results[0][0]['sequence']: # make sure that the new word chosen by CamemBERT is different from the one in the original text
        print(results[0][random.randint(1,len(results)-1)]['sequence'])
    else:
        print(results[0][0]['sequence'])
else :
    if text.replace("\n","") == results[0]['sequence']: # idem
        print(results[random.randint(1,len(results)-1)]['sequence'])
    else:
        print(results[0]['sequence'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce dimanche 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Wall time: 121 ms


## 2. Rétro-traduction

### 2.1. Marian (Helsinki-NLP models)
Full doc : https://huggingface.co/transformers/model_doc/marian.html 

In [9]:
from transformers import pipeline

In [10]:
%%time
fwd_scr = 'fr'  # source language
fwd_trg = 'en'  # target language
rev_scr = 'en'  # source language
rev_trg = 'fr'  # target language

forward_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{fwd_scr}-{fwd_trg}', f'Helsinki-NLP/opus-mt-{fwd_scr}-{fwd_trg}')
reverse_translator = pipeline("translation", f'Helsinki-NLP/opus-mt-{rev_scr}-{rev_trg}', f'Helsinki-NLP/opus-mt-{rev_scr}-{rev_trg}')
new_text = reverse_translator(forward_translator(text)[0]['translation_text'])

# original text
print(text)

# new text
print(new_text[0]['translation_text'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, 19 ans, a fait un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, leader mondial, en quart de finale du tournoi WTT Champions à Macao, en Chine.
Wall time: 7.47 s


### 2.2.2 M2M100
Full doc : https://huggingface.co/transformers/model_doc/m2m_100.html  
Sumamry : a single model with about 400M parameters (proposed by Facebook) that manages about 100 languages  

Two implementations are available: one based on the HF pipeline function, the other using the m2m100 model card code.  
The pipeline seems to take much longer to run. So it may not be the best approach to take.

In [ ]:
from transformers import pipeline

In [18]:
%%time

forward_translator = pipeline('translation', 'facebook/m2m100_418M', src_lang=fwd_scr, tgt_lang=fwd_trg)
reverse_translator = pipeline('translation', 'facebook/m2m100_418M', src_lang=rev_scr, tgt_lang=rev_trg)
new_text = reverse_translator(forward_translator(text)[0]['translation_text'])

# original text
print(text)

# new text
print(new_text[0]['translation_text'])

Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, âgé de 19 ans, a fait un résultat vendredi 21 avril en défaisant le chinois Fan Zhendong, numéro un mondial, dans les quarts de finale du Tournoi des champions du WTT à Macao, en Chine.
Wall time: 43.7 s


In [16]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [17]:
%%time
# forward_translator
tokenizer.src_lang = "fr"
encoded = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True
                       
# forward_translator
src_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
tokenizer.src_lang = "en"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("fr"))

# original text
print(text)

# new text
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

C:\Users\lbourdois\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Le pongiste français Alexis Lebrun, 19 ans, a réalisé un exploit ce vendredi 21 avril en battant le Chinois Fan Zhendong, numéro 1 mondial, en quarts de finale du tournoi WTT Champions de Macao, en Chine.
Le pongiste français Alexis Lebrun, âgé de 19 ans, a fait un résultat vendredi 21 avril en défaisant le chinois Fan Zhendong, numéro un mondial, dans les quarts de finale du Tournoi des champions du WTT à Macao, en Chine.
Wall time: 24.1 s
